# Web data from __[telekom.de](https://www.telekom.de/hilfe)__

## Overview

### Function
`extract_and_write_data(directory_path, web_data_file, filtered_file, inspecting_file)`

### Description
This function processes `.txt` files in a specified directory to extract relevant information and write it to different output files based on their content. It categorizes the files into three categories:
1. **Files with valid questions and answers**: Written to `web_data.txt`.
2. **Files that could not be processed**: Written to `filtered.txt`.
3. **Files with questions containing `?` but not ending with `?`**: Written to `inspecting.txt`.

### Inputs
- **`directory_path`**: Path to the directory containing `.txt` files.
- **`web_data_file`**: Path to the output file for valid questions and answers.
- **`filtered_file`**: Path to the output file for files that couldn't be processed.
- **`inspecting_file`**: Path to the output file for questions containing `?` but not ending with `?`.

### Outputs
- **`web_data.txt`**: Contains the valid question and answer data.
- **`filtered.txt`**: Contains filenames of files that couldn't be processed.
- **`inspecting.txt`**: Contains questions with `?` but not ending with `?` along with their details.


In [31]:
import re
import os

# Define fixed paths for the input directory and output files
directory_path = "/Users/taha/Desktop/scrapeV2/output_folder"
web_data_directory = "web_data"
web_data_file = os.path.join(web_data_directory, "web_data.txt")
filtered_file = os.path.join(web_data_directory, "filtered.txt")
inspecting_file = os.path.join(web_data_directory, "inspecting.txt")

def extract_and_write_data(directory_path, web_data_file, filtered_file, inspecting_file):
    """
    Extracts and categorizes data from .txt files in the specified directory.
    
    Parameters:
    - directory_path (str): Path to the directory containing .txt files.
    - web_data_file (str): Path to the output file for valid questions and answers.
    - filtered_file (str): Path to the output file for files that couldn't be processed.
    - inspecting_file (str): Path to the output file for questions containing `?` but not ending with `?`.
    
    Outputs:
    - Writes valid questions and answers to web_data_file.
    - Writes filenames of unprocessed files to filtered_file.
    - Writes questions with `?` but not ending with `?` to inspecting_file.
    """

    # Regex pattern to find the section starting with "...Telekom" and ending with two spaces
    pattern = r"\.\.\.Telekom.*?\s{2}"
    
    # Initialize counters and lists to keep track of files and their processing
    total_files = 0
    processed_files = 0
    processed_files_data = []
    unprocessed_files = []
    inspecting_files = []

    def process_file(filename):
        """
        Processes each .txt file to extract and categorize content based on patterns.
        
        Parameters:
        - filename (str): The name of the file to be processed.
        """
        nonlocal processed_files
        file_path = os.path.join(directory_path, filename)
        try:
            with open(file_path, "r", encoding="utf-8") as source:
                content = source.read()
                match = re.search(pattern, content, re.DOTALL)
                if match:
                    processed_files += 1
                    matched_text = match.group(0)
                    start_index = match.end()

                    # Clean and format the matched text
                    cleaned_text = re.sub(r'\.\.\.+', '\n', matched_text.strip(".").strip())
                    
                    # Extract the text after the matched pattern
                    post_pattern_text = content[start_index:]
                    
                    # Find paragraphs separated by multiple newlines
                    paragraph_pattern = r'([^\n]+(?:\n[^\n]+)*)(?:\n{2,})'
                    paragraphs = re.findall(paragraph_pattern, post_pattern_text)
                    
                    if len(paragraphs) >= 2:
                        question = paragraphs[0].strip()
                        answer = paragraphs[1].strip()
                        
                        if question.endswith('?'):
                            processed_files_data.append((filename, cleaned_text, question, answer))
                        else:
                            if '?' in question:
                                inspecting_files.append(f"File: {filename}\nNavigation:\n{cleaned_text}\n\nQuestion: {question}\n\nAnswer: {answer}\n")
                    else:
                        unprocessed_files.append(filename)
                else:
                    unprocessed_files.append(filename)
        except Exception as e:
            unprocessed_files.append(filename)

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Directory {directory_path} does not exist.")
        return

    # Create web_data directory if it does not exist
    os.makedirs(web_data_directory, exist_ok=True)

    # Process each .txt file in the directory
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith(".txt"):
            total_files += 1
            process_file(filename)

    # Write valid question and answer data to web_data_file
    with open(web_data_file, "w", encoding="utf-8") as web_data_txt:
        for filename, navigation, question, answer in sorted(processed_files_data, key=lambda x: x[0]):
            web_data_txt.write(f"File: {filename}\n")
            web_data_txt.write(f"Navigation:\n{navigation}\n\n")
            web_data_txt.write(f"Question: {question}\n\n")
            web_data_txt.write(f"Answer: {answer}\n")
            web_data_txt.write("\n" + "="*40 + "\n")

    # Write filenames of unprocessed files to filtered_file
    with open(filtered_file, "w", encoding="utf-8") as filtered_txt:
        for filename in sorted(unprocessed_files):
            filtered_txt.write(f"{filename}\n")

    # Write questions with `?` but not ending with `?` to inspecting_file
    with open(inspecting_file, "w", encoding="utf-8") as inspecting_txt:
        inspecting_txt.writelines(inspecting_files)

    # Calculate numbers for print statements
    num_unprocessed_files = len(unprocessed_files)
    num_inspecting_files = len(inspecting_files)
    num_processed_files = len(processed_files_data)
    missing_files = total_files - (num_processed_files + num_unprocessed_files + num_inspecting_files)

    # Print summary of the processing
    print("Process completed.")
    print(f"Total number of .txt files in the folder: {total_files}")
    print(f"Number of .txt files processed and written to {web_data_file}: {num_processed_files}")
    print(f"Unprocessed files have been written to {filtered_file}. Number of unprocessed files: {num_unprocessed_files}")
    print(f"Files with questions containing '?' but not ending with '?' have been written to {inspecting_file}. Number of inspecting files: {num_inspecting_files}")
    print(f"Number of missing or unaccounted files: {missing_files}")

# Call the function with the specified parameters
extract_and_write_data(directory_path, web_data_file, filtered_file, inspecting_file)


Process completed.
Total number of .txt files in the folder: 2595
Number of .txt files processed and written to web_data/web_data.txt: 1048
Unprocessed files have been written to web_data/filtered.txt. Number of unprocessed files: 919
Files with questions containing '?' but not ending with '?' have been written to web_data/inspecting.txt. Number of inspecting files: 69
Number of missing or unaccounted files: 559


## Download all pdf files of __[telekom.de/hilfe](https://www.telekom.de/hilfe)__

In [2]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

# PDF dosyalarını indireceğimiz dizin
DOWNLOAD_DIR = 'pdfs'
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

# Başlangıç URL'si
START_URL = 'https://www.telekom.de/hilfe'

# İndirilen PDF dosyalarının isimlerini saklayacak set
downloaded_files = set()

# PDF dosyalarının bağlantılarını bulmak için fonksiyon
def find_pdfs(url):
    pdf_links = []
    try:
        response = requests.get(url)
        # Check if the content type is HTML
        if response.headers['Content-Type'].lower().startswith('text/html'):
            # Try different parsers if 'html.parser' fails
            try:
                soup = BeautifulSoup(response.text, 'html.parser')
            except FeatureNotFound:
                soup = BeautifulSoup(response.text, 'lxml')
            except Exception as e:
                print(f'Hata: {url} içerik pars edilemedi. {str(e)}')
                return pdf_links

            for a in soup.find_all('a', href=True):
                href = a['href']
                full_url = urljoin(url, href)
                if full_url.lower().endswith('.pdf'):
                    pdf_links.append(full_url)
        else:
            print(f'Hata: {url} HTML değil.')
    except Exception as e:
        print(f'Hata: {url} indirilemedi. {str(e)}')
    
    return pdf_links

# PDF dosyalarını indir
def download_pdf(url):
    try:
        filename = url.split('/')[-1]
        if filename in downloaded_files:
            print(f'Zaten indirildi: {filename}')
            return
        
        response = requests.get(url)
        if response.status_code == 200:
            file_path = os.path.join(DOWNLOAD_DIR, filename)
            with open(file_path, 'wb') as f:
                f.write(response.content)
            downloaded_files.add(filename)
            print(f'İndirildi: {file_path}')
        else:
            print(f'Hata: {url} indirilemedi. Status code: {response.status_code}')
    except Exception as e:
        print(f'Hata: {url} indirilemedi. {str(e)}')

# Web sitesini tarayan fonksiyon
def crawl_site(start_url):
    urls_to_visit = [start_url]
    visited_urls = set()
    while urls_to_visit:
        url = urls_to_visit.pop(0)
        if url in visited_urls:
            continue
        visited_urls.add(url)
        print(f'Taranıyor: {url}')
        pdf_links = find_pdfs(url)
        for link in pdf_links:
            download_pdf(link)
        try:
            response = requests.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                for a in soup.find_all('a', href=True):
                    href = a['href']
                    full_url = urljoin(url, href)
                    if full_url.startswith(start_url) and full_url not in visited_urls:
                        urls_to_visit.append(full_url)
        except Exception as e:
            print(f'Hata: {url} taranamadı. {str(e)}')

# Başlat
if __name__ == '__main__':
    crawl_site(START_URL)


Taranıyor: https://www.telekom.de/hilfe
İndirildi: pdfs/051.pdf
İndirildi: pdfs/000.pdf
Taranıyor: https://www.telekom.de/hilfe/zur-telekom-wechseln
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/magentaservice
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/vertrag-rechnung
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/mobilfunk
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/internet-telefonie
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/tv
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/magenta-eins
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/apps-dienste
Zaten indirildi: 051.pdf
Zaten indirildi: 000.pdf
Taranıyor: https://www.telekom.de/hilfe/geraete-zubehoer
Zaten indirildi: 051.pdf